In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from scipy.stats import norm
from scipy.optimize import brentq
from scipy import interpolate
from scipy.integrate import quad


In [2]:
# Import filescd spx_prices = pd.read_csv('SPX_options.csv')
zero_rates = pd.read_csv('zero_rates_20201201.csv')

### Black-Scholes Model:
Under the Black-Scholes model, the underlying asset price dynamic follows a geometric Brownian motion, expressed as the following stochastic differential equation:

$$dS_t = rS_tdt + \sigma S_tdW_t^\mathbb{Q}$$

#### Given the payoff function: 
$$\begin{aligned} \mathcal{H}(S_T) &= {S_T}^{1/3} + 1.5 × log(S_T) + 10.0 \\
&= S_0^{\frac{1}{3}}exp\bigg(\frac{1}{3}(r-\frac{\sigma^2}{2})T +\frac{1}{3}σ\sqrt{T}x\bigg) +1.5\bigg(log(S_0)+(r-\frac{\sigma^2}{2})T +σ\sqrt{T}x\bigg) + 10 \end{aligned}$$

we derive the derivative pricing as follows:

$$\begin{aligned} V_c &= e^{-rT}\mathbb{E}_t\bigg[\mathcal{H}(S_T)\bigg]\\
&= e^{-rT}\bigg(S_0^{\frac{1}{3}}exp((\frac{1}{3}r - \frac{1}{9}\sigma^2)T) + 1.5 (log(S_0)+(r-\frac{\sigma^2}{2})T)+10\bigg)
\end{aligned}$$



In [3]:
def BlackScholesDerivative(S, r, sigma, T):
    
    n = 1/3
    
    # Term1_valuation = np.exp(-r*T)*(S**n)*(np.exp((n*rT - (n*sigma**2)*T)/3))
    Term1_valuation = np.exp(-r*T)*(S**n)*(np.exp((n*r*T - (n*sigma)**2*T)))
    Term2_valuation = np.exp(-r*T) * 1.5* (np.log(S) + (r - 0.5*sigma**2)*T)
    Term3_valuation = np.exp(-r*T)*10.0
                              
    BS_Valuation = Term1_valuation + Term2_valuation + Term3_valuation
                    
    return BS_Valuation

### Bachelier Model:

Under the Bachelier model, the underlying asset price dynamic follows an arithmetic Brownian motion, expressed as the following stochastic differential equation:

$$dS_t = \sigma S_0dW_t^\mathbb{Q}$$

#### Given the payoff function: 
$$\begin{aligned} \mathcal{H}(S_T) &= {S_T}^{1/3} + 1.5 × log(S_T) + 10.0 \\
&= S_0^{\frac{1}{3}}(1 + \sigma \sqrt{T}x)^{\frac{1}{3}} + 1.5\times log(S_0(1 + \sigma \sqrt{T}x)) + 10\end{aligned}$$

By Binomial expansion, we express

$$S_0^{\frac{1}{3}}(1 + \sigma \sqrt{T}x)^{\frac{1}{3}} = S_0^{\frac{1}{3}}\bigg(1 + \frac{1}{3}\frac{\sigma \sqrt{T}}{S_0}x - \frac{1}{9}\bigg(\frac{\sigma \sqrt{T}}{S_0}\bigg)^2x^2+...\bigg)$$

we derive the derivative pricing as follows:

$$\begin{aligned} V_c &= e^{-rT}\mathbb{E}_t\bigg[\mathcal{H}(S_T)\bigg]\\
&= e^{-rT}\bigg(S_0^{\frac{1}{3}}\bigg(1 - \frac{1}{9}(\frac{\sigma}{S_0})^2T\bigg) + 1.5\bigg(log(S_0) + \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty}log(1+\sigma\sqrt{T}x) e^{-\frac{x^2}{2}}dx\bigg) + 10 \bigg)
\end{aligned}$$

In [4]:
#------------------------------------------------------------------------------
#Bachelier Model

def integrand(x, sigma, T):
  return np.log(1+sigma*np.sqrt(T)*x)*np.exp(-(x**2)/2)

def BachelierDerivative(S, r, sigma, T):
  n = 1/3
  
  #-----To be confirmed-----#
  Term1_valuation = (S**n)*(1-n**2*(sigma/S)**2*T)
  Term2_valuation = 1.5*np.log(S)
  Term3_valuation = 1.5*1/np.sqrt(2*np.pi)*quad(lambda x: integrand(x, sigma, T), 0,5000)[0] #pls reconfirm if this is correct
  Term4_valuation = 10.0
  #-----To be confirmed-----#
                              
  B_Valuation = np.exp(-r*T)*(Term1_valuation + Term2_valuation + Term3_valuation + Term4_valuation)
                    
  return B_Valuation
#------------------------------------------------------------------------------

### Static-replication of European payoff: (1st Derivative Contract)

$$ {S_T}^{1/3} + 1.5 × log(S_T) + 10.0 $$



In [5]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [6]:
def SABR(F, K, T, alpha, beta, rho, nu):
  X = K
  # if K is at-the-money-forward
  if abs(F - K) < 1e-12:
    numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
    numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
    numer3 = ((2 - 3*rho*rho)/24)*nu*nu
    VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
    sabrsigma = VolAtm
  
  else:
    z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
    zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
    numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
    numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
    numer3 = ((2 - 3*rho*rho)/24)*nu*nu
    numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
    denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
    denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
    denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
    sabrsigma = numer/denom
    
  return sabrsigma

def SABRCall(S, K, r, alpha, beta, rho, nu, T):
  sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
  return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
  sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
  return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
  price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
  return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
  price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
  return price

def CMintegrand(K, S, r, T, alpha, beta, rho, nu, fxn):
  H_doublederiv = -2/9*(K)**(-5/3) - 1.5/(K**2)
  return H_doublederiv * fxn(K, S, r, T, alpha, beta, rho, nu)

In [7]:
def Carr_and_Madan(PK, CK, S, r, T, alpha, beta, rho, nu):
  F = S * np.exp(r*T)
  print(F)
  
  hF=F**(1/3) + 1.5*np.log(F) + 10.0
  
  f1 = quad(lambda x: CMintegrand(x, S, r, T, alpha, beta, rho, nu, PK), 10, F)[0]
  f2 = quad(lambda x: CMintegrand(x, S, r, T, alpha, beta, rho, nu, CK), F, 5500)[0]

  CM_Valuation = hF*np.exp(-r*T) + f1 + f2
    
  return CM_Valuation

## Instrument 2

### Static Replication of European Payoff (2nd Derivative Contract)

Given the payoff of the second derivative is as follows:
$$ \sigma^2_{MF}T  = E \bigg[ \int_{0}^{T} \sigma^2_{t} \,dt \bigg]  $$

Under Black-Scholes and Bachelier Models, the volatility is assumed to be deterministic and constant across time. Therefore, under these models, the price of the derivative is given by:

$$\begin{aligned}V_c &= \mathbb{E}_t\bigg[\int_0^T \sigma_{Model}^2 dt\bigg] \\
&= \sigma^2_{Model} T \end{aligned}$$

The pricing formula using static replication method is given as follows:
$$\begin{aligned}V_c &= \mathbb{E}_t\bigg[\int_0^T \sigma_{t}^2 dt\bigg] \\
&= 2e^{rT}\bigg(\int_0^F\frac{P(K)}{K^2}dK + \int_F^\infty\frac{C(K)}{K^2}dK\bigg) \end{aligned}$$

In [8]:
def evar(S, r, T, alpha, beta, rho, nu):
  F = S*np.exp(r*T)
  I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 10, F)
  I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
  E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])

  return E_var

## Implementation

In [9]:
# Basic Params
days_to_expiry = (pd.Timestamp('2021-01-15') - pd.Timestamp('2020-12-01')).days
def get_zero_rate(df_zr, days_to_expiry):
    days_maturity1, rate1 = df_zr[df_zr['days'] < days_to_expiry].iloc[-1,1:]
    days_maturity2, rate2 = df_zr[df_zr['days'] > days_to_expiry].iloc[0,1:]
    
    return rate1 + (days_to_expiry - days_maturity1)/(days_maturity2 - days_maturity1)*(rate2 - rate1)
r = get_zero_rate(zero_rates, days_to_expiry)/100
T = days_to_expiry/365

# Underlying spot prices
S_spx = 3662.45
S_spy = 366.02

# Underlying ATM Volatilities (From part 2)
ATM_vol_spx = 0.184829
ATM_vol_spy = 0.177927

# SABR Parameters (From part 2)
alpha = [1.816504, 0.871586]
beta = 0.7
rho = [-0.404302, -0.493586]
nu = [2.790158, 2.800048]


In [10]:
prices = [S_spx, S_spy]
volas = [ATM_vol_spx, ATM_vol_spy]
deriv1_prices = []

for idx, itm in enumerate(['SPX', 'SPY']):
  deriv_dict = {}
  deriv_dict['Underlying'] = itm
  deriv_dict['Black-Scholes'] = BlackScholesDerivative(prices[idx], r, volas[idx], T)
  deriv_dict['Bachelier'] = BachelierDerivative(prices[idx], r, volas[idx], T)
  deriv_dict['Static Replication'] = Carr_and_Madan(sabrputintegrand, sabrcallintegrand, prices[idx], r, T, alpha[idx], beta, rho[idx], nu[idx])

  deriv1_prices.append(deriv_dict)

df_deriv1 = pd.DataFrame(deriv1_prices)
df_deriv1

3663.3762493669747
366.11256803322914


,Underlying,Black-Scholes,Bachelier,Static Replication
0,SPX,37.704907,37.750944,37.715275
1,SPY,25.995631,26.036679,26.001658


In [11]:
deriv2_prices = []

for idx, itm in enumerate(['SPX', 'SPY']):
  deriv_dict = {}
  deriv_dict['Underlying'] = itm
  deriv_dict['Black-Scholes'] = volas[idx]**2 * T
  deriv_dict['Bachelier'] = volas[idx]**2 * T
  deriv_dict['Static Replication'] = evar(prices[idx], r, T, alpha[idx], beta, rho[idx], nu[idx])

  deriv2_prices.append(deriv_dict)

df_deriv2 = pd.DataFrame(deriv2_prices)
df_deriv2

,Underlying,Black-Scholes,Bachelier,Static Replication
0,SPX,0.004212,0.004212,0.006306
1,SPY,0.003903,0.003903,0.005765
